In [10]:
import sys, os
from dateutil import parser
import pandas as pd, numpy as np 
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import warnings
warnings.simplefilter(action='ignore')

# Setting directory
mypath_Al = '/home/oline/Documents/Berkeley/lbl/room131/Monitor/Al_rapports/'

# Loading efficiency calibration
C05 = np.load('C05.npy')
C10 = np.load('C10.npy')
C15 = np.load('C15.npy')
C18 = np.load('C18.npy')
C22 = np.load('C22.npy')
C30 = np.load('C30.npy')
C40 = np.load('C40.npy')
C50 = np.load('C50.npy')
C60 = np.load('C60.npy')

In [11]:
"""
Read all information in directory
"""

def read_Fits_Rapport(path):
    live_time, date_spec, sample, distance = [], [], [], []
    fitsE, fitsN,  fitsN_unc = [], [], []
    
    for filename in os.listdir(path):
        with open(path + filename, 'r') as infile:
            lines = infile.readlines()
            fitz_energy, fitz_peak_area, fitz_peak_area_unc = get_Fitz_information(lines[33:])
            for j in range(len(fitz_energy)):
                fitsE.append(fitz_energy[j])
                fitsN.append(fitz_peak_area[j])
                fitsN_unc.append(fitz_peak_area_unc[j]/100)                      # Relative error 
                date_time = lines[20].split()[1] + ' '+ lines[20].split()[2]
                live_time.append(np.float(lines[21].split()[-1]))
                date_spec.append(str(parser.parse(date_time)))
                sample.append(filename[11:15])
                distance.append(filename[16:18])

    df = pd.DataFrame({'sample_ID': sample, 'distance [cm]': distance , 'fits_energy': fitsE, 'Peak_area': fitsN, 'Peak_area_unc': fitsN_unc, 'live_time [s]': live_time, 'date_time': date_spec })

    return df


def get_Fitz_information(lines):
    """ Called in the read_files function to obtain energy and peak area
    """
    energy, peak_area, peak_area_unc = [], [], []
    
    for line in lines:
        try:
            energy.append(float(line.split()[0]))
            peak_area.append(float(line.split()[5]))
            peak_area_unc.append(float(line.split()[6]))
        except ValueError:
            pass
        except IndexError:
            pass
        
    return energy, peak_area, peak_area_unc

In [32]:
"""
Prepare pandas dataframe for analysis
"""

def make_empty_str(title, df, length):
    df[title] = [''] * length
    return df

def make_empty_float(title, df, length):
    df[title] = np.array([np.NaN] * length)
    return df

def expand_frame(df):
    df = make_empty_str('Isotope', df, len(df))
    df = make_empty_str('Com', df, len(df))
    df = make_empty_str('Isotope Decaytime', df, len(df))
    df = make_empty_float('NucDat Energy [kEv]', df, len(df))
    df = make_empty_float('NucDat Energy unc [kEv]', df, len(df))
    df = make_empty_float('NucDat Intensity [%]', df, len(df))
    df = make_empty_float('NucDat Intensity unc [%]', df, len(df))
    return df

def information(df, Index, Isotope, Comment, Energy, Energy_unc, Intensity, Intensity_unc):
    df['Isotope'][Index] = Isotope
    df['Com'][Index] = Comment
    df['NucDat Energy [kEv]'][Index] =  Energy
    df['NucDat Energy unc [kEv]'][Index] =  Energy_unc
    df['NucDat Intensity [%]'][Index] = Intensity
    df['NucDat Intensity unc [%]'][Index] = Intensity_unc
    return df

def check_if_previously_identified(df1,df2):
    df_1 = df1.copy()
    df_2 = df2.copy()
    for i in range(len(df_1)):
        for j in range(len(df_2)):
            if abs(df_1['fits_energy'][i] - df_2['fits_energy'][j]) < 0.21:
                df_2['Isotope'][j] = df_1['Isotope'][i]
                df_2['Com'][j] = df_1['Com'][i]
                df_2['NucDat Energy [kEv]'][j] = df_1['NucDat Energy [kEv]'][i]
                df_2['NucDat Energy unc [kEv]'][j] = df_1['NucDat Energy unc [kEv]'][i]
                df_2['NucDat Intensity [%]'][j] = df_1['NucDat Intensity [%]'][i]
                df_2['NucDat Intensity unc [%]'][j] = df_1['NucDat Intensity unc [%]'][i]
    return df_2

def reorganize_columns(df_):
    df = df_.copy()
    df = df[['sample_ID',
             'distance [cm]',
             'Isotope',
             'Com',
             'fits_energy',
             'NucDat Energy [kEv]',
             'NucDat Energy unc [kEv]',
             'NucDat Intensity [%]',
             'NucDat Intensity unc [%]',
             'Peak_area',
             'Peak_area_unc',
             'live_time [s]',
             'date_time',
             'Isotope Decaytime']]
    return df

In [40]:
"""
Prepering Al Database 
"""

Al = read_Fits_Rapport(mypath_Al)

Al_1_18 = Al[Al['sample_ID'] == 'Al-1'].reset_index()
Al_3_16 = Al[Al['sample_ID'] == 'Al-3'].reset_index()

Al_1_18 = expand_frame(Al_1_18)
Al_3_16 = expand_frame(Al_3_16)


In [41]:
Al_1_18  = information(Al_1_18, 0,'Background',  '-', '','','','')
Al_1_18  = information(Al_1_18, 1,'Background',  '-', '','','','')
Al_1_18  = information(Al_1_18, 2,'Background',  '84.6. 56Cr, but E=26 gamma not found at I = 44.6, T_half = 5.94 min',  '','','','')
Al_1_18  = information(Al_1_18, 3,'Background',  '87.2', '','','','')
Al_1_18  = information(Al_1_18, 3,'Annihilation Energy',  '-', '','','','')
Al_1_18  = information(Al_1_18, 3,'Background',  '87.2', '','','','')

In [42]:
Al_1_18 = reorganize_columns(Al_1_18)
Al_1_18

,sample_ID,distance [cm],Isotope,Com,fits_energy,NucDat Energy [kEv],NucDat Energy unc [kEv],NucDat Intensity [%],NucDat Intensity unc [%],Peak_area,Peak_area_unc,live_time [s],date_time,Isotope Decaytime
0,Al-1,18,Background,-,72.5,,,,,5524.0,0.149,226.0,2019-02-26 04:59:00,
1,Al-1,18,,,74.7,NaN,NaN,NaN,NaN,10995.0,0.076,226.0,2019-02-26 04:59:00,
2,Al-1,18,,,84.6,NaN,NaN,NaN,NaN,4449.0,0.106,226.0,2019-02-26 04:59:00,
3,Al-1,18,,,87.0,NaN,NaN,NaN,NaN,1347.0,0.331,226.0,2019-02-26 04:59:00,
4,Al-1,18,,,511.5,NaN,NaN,NaN,NaN,33560.0,0.014,226.0,2019-02-26 04:59:00,
5,Al-1,18,,,514.8,NaN,NaN,NaN,NaN,1079.0,0.178,226.0,2019-02-26 04:59:00,
6,Al-1,18,,,844.5,NaN,NaN,NaN,NaN,848.0,0.168,226.0,2019-02-26 04:59:00,
7,Al-1,18,,,847.5,NaN,NaN,NaN,NaN,1723.0,0.092,226.0,2019-02-26 04:59:00,
8,Al-1,18,,,858.5,NaN,NaN,NaN,NaN,356.0,0.389,226.0,2019-02-26 04:59:00,
9,Al-1,18,,,932.2,NaN,NaN,NaN,NaN,411.0,0.334,226.0,2019-02-26 04:59:00,
